In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
from keras.utils.vis_utils import plot_model
from keras.models import load_model
import modules.deep_model as DeepModel
import modules.model as Model 
import tensorflow as tf
import warnings
import numpy as np
import fasttext
import os

warnings.filterwarnings('ignore')

# Sử dụng CNN1D của Keras
* Phần này chúng ta sẽ thử sử dụng **word vectors** của Facebook mà chúng ta đã dùng để trực quan hóa dữ liệu lên không gian 3D ở project 2, mục đích ta muốn biết liệu các **word vectors** này có hoạt động hiệu quả trên dataset của chúng ta không.
* Ở CNN, chúng ta sử dụng phép **convolution** [tích chập] giữa **window** [cửa số lọc] có kích thước 3x3 hoặc 5x5,.. - duyệt qua từng vùng tương ứng trên ảnh để tìm hiểu các đặc trưng phức tạp của hình ảnh.
* Vậy giả sử từng comment bao gồm $n$ từ, ta có thể dùng **fasttext 100 dimmensions** để biểu diễn từng từ trong comment sang một không gian vector 100 dimensions, như vậy lúc này một comment của chúng ta được chuyển đổi thành một ma trận có kích thước $100 \times n$.
* Nhưng lúc này các comment của chúng ta sẽ không thống nhất về chiều dài, nên ta sẽ ép chúng về chung một chiều dài $m$ nào đó - trong code bên dưới là 100. Và với những comment có $n < m$, ta sẽ thay thế phần bù còn lại là một vector 0.
* Khi áp dụng CNN cho bài toán NLP, người ta hay áp dụng window có kích thước từ 2 đến 5. Giả sử window có kích thước là 5 thì chúng ta đang trượt qua comment và tính toán cho 5 từ một lúc, nó hoạt động như cách chúng ta code cho N-Grams ở project 2 vậy.
* Về mặt thời gian, thì CNN hoạt động nhanh hơn so với LSTM, tuy nhiên về mặt hiệu năng, khả năng cao kết quả cho ra không chính xác, vì mặc dù ta đại diện cho một từ là một vector 100 dimensions nhưng chưa có điều gì đủ để chứng minh các vector này phù hợp với tập dữ liệu của chúng ta. Nên việc ta áp dụng ở đây đơn thuần là để kiểm tra hơn là việc hi vọng nó hiệu quả.
* Như vậy chiến lược của chúng ta sẽ như sau:
  * **Bước 1**: Ta sẽ định nghĩa một chiều dài cố định cho từng comment trong training data, như project 2 ta biết được rằng một comment có khoảng gần 100 từ, như vậy lúc này một comment của chúng ta sau quá trình **Embedding** sẽ là một ma trận có kích thước $100 \times 100$. Và ta cập nhật `weights` cho **Embedding layer** bằng chính các comment mà mỗi comment là một ma trận.
  * **Bước 2**: Tiếp theo, ta sẽ định nghĩa window của chúng ta có kích thước là 5, ta tiến hành convolution giữa **Input layer** và **Embedding layer**, sau đó ta cho kết quả đi qua một **MaxPooling layer** có kích thước là 5, 5 giá trị lân cận chọn ra giá trị lớn nhất.
  * **Bước 3**: Ta sử dụng một **GlobalPoolingMax** để gộp các output ở bước trước lại, việc này tương tự như ta dùng **Flatten layer** vậy.
  * **Bước 4**: Ta định nghĩa output layer gồm 2 neurons và activation function là softmax.
  * **Bước 5**: Ta compile model bằng **binary cross-entropy** loss function với optimizer là **Adam** và metric là **accuracy**.

* Tất cả ý tưỡng trên được cấu hình trong class `SentimentCNN1D` trong file `deep_model.py`. 

Đọc dữ liệu train và test.

In [2]:
X_train, y_train = Model.loadData("./data/data/train")
X_train = X_train.fillna("")

X_test, y_test = Model.loadData("./data/data/test")
X_test = X_test.fillna("")

display(X_train, y_train)
display(X_test, y_test)

,raw_comment,normalize_comment,emoji_decode
0,Shop gói hàng chắc chắn. Giai hàng nhan...,gói hàng chắc chắn giai hàng nhanh màu...,
1,Chất liệu vải thì khỏi bàn. Chỉ mỗi...,chất liệu vải khỏi bàn chỉ mỗi may đ...,
2,kẹp đẹp dã mannnnnnnnnnnnnnnnnnnnnnnnnnnnnn...,kẹp đẹp dã,
3,Áo ko đúng size\nSiza xl mà như size m v,áo không đúng size xl mà như size,
4,Với giá thì sp khá tốt\nNhưng lông ho...,giá sản phẩm khá tốt nhưng lông hơi ...,
...,...,...,...
11356,Áo cũng ok nhưng vải nóng quá,áo cũng ok nhưng vải nóng quá,
11357,quần đẹp mà áo thì hơi mỏng,quần đẹp mà áo hơi mỏng,
11358,"Khóa bị kẹt, kéo qua lại k mượt. Nói ...",kẹt kéo qua không mượt nói chung tạm o...,
11359,Đặt hàng lần thứ nhất shop giao thiê...,đặt hàng lần thứ nhất giao thiếu mo...,


,label
0,1
1,0
2,1
3,0
4,0
...,...
11356,0
11357,1
11358,0
11359,0


,raw_comment,normalize_comment,emoji_decode
0,"Đẹp , nhẹ , đường may nhiều đầu chỉ ...",đẹp nhẹ đường may nhiều đầu chỉ các...,
1,Giao hàng nhanh\nĐóng gói sản phẩm cẩn...,giao hàng nhanh đóng gói sản phẩm cẩn ...,
2,Quần đẹp lắm \nShop thân thiện \nNhiê...,quần đẹp lắm thân thiện nhiều o,
3,"Uiii, xinhhhhh lắmmmmmm luônnnnn, ship nhan...",ui xinh lắm luôn ship nhanh nưa,
4,Chất lượng tốt giao hàng nhanh nhân v...,chất lượng tốt giao hàng nhanh nhân v...,thumbsup thumbsup thumbsup thumbsup thumbsup
...,...,...,...
2836,Chất hơi bị mềm quá. Nc vẫn đc.,chất hơi mềm quá nói chuyện vẫn đư...,
2837,Nhìn hình tưởng quần dày có thể mạ...,nhìn hình tưởng quần dày có thể mạ...,
2838,Sản phẩm bị lủng rách nhiều chổ mua ...,sản phẩm lủng rách nhiều chổ mua cái...,
2839,Cay vừa mua 180k hôm sau giảm 120k,cay vừa mua không hôm sau giảm không,


,label
0,0
1,1
2,1
3,1
4,1
...,...
2836,1
2837,0
2838,0
2839,1


Load model **fast text** cho tiếng Việt để tiến hàng chuyển đổi các word thành các **vector 100 dimensions**.

In [4]:
# ft = fasttext.load_model("./cc.vi.100.bin")

Định nghĩa các Hyper-Parameters.

In [7]:
hyperparams_1 = {
    'pX': X_train['normalize_comment'], 
    'py': y_train, 
    'pfasttext': ft, 
    'pno_neurons': 10,
    'pno_filters': 5, 
    'pnum_words': 3000, 
    'pseq_length': 100,
    'pembedding_dim': 100,
    'pbatch_size': 32, 
    'pepochs': 10,
    'psave_path': './models/cnn1d_model_comments_1.h5'
}

model_1 = DeepModel.SentimentCNN1D()
model_1.define(**hyperparams_1)
model_1.model.summary()
plot_model(model_1.model, to_file='./images/cnn1d_model_comments_1.png', show_shapes=True, show_layer_names=True)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 100)]             0         
                                                                 
 embedding (Embedding)       (None, 100, 100)          356800    
                                                                 
 conv1d (Conv1D)             (None, 96, 10)            5010      
                                                                 
 max_pooling1d (MaxPooling1D  (None, 19, 10)           0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 15, 10)            510       
                                                                 
 global_max_pooling1d (Globa  (None, 10)               0         
 lMaxPooling1D)                                              

In [8]:
model_1.fit(**hyperparams_1)

Epoch 1/10
320/320 [==============================] - 3s 6ms/step - loss: 0.6764 - accuracy: 0.5709 - val_loss: 0.6464 - val_accuracy: 0.6288
Epoch 2/10
320/320 [==============================] - 2s 6ms/step - loss: 0.6281 - accuracy: 0.6346 - val_loss: 0.6070 - val_accuracy: 0.6508
Epoch 3/10
320/320 [==============================] - 2s 6ms/step - loss: 0.6052 - accuracy: 0.6513 - val_loss: 0.6058 - val_accuracy: 0.6420
Epoch 4/10
320/320 [==============================] - 2s 6ms/step - loss: 0.5925 - accuracy: 0.6629 - val_loss: 0.5998 - val_accuracy: 0.6438
Epoch 5/10
320/320 [==============================] - 2s 6ms/step - loss: 0.5826 - accuracy: 0.6674 - val_loss: 0.6000 - val_accuracy: 0.6447
Epoch 6/10
320/320 [==============================] - 2s 6ms/step - loss: 0.5768 - accuracy: 0.6759 - val_loss: 0.6007 - val_accuracy: 0.6491
Epoch 7/10
320/320 [==============================] - 2s 6ms/step - loss: 0.5701 - accuracy: 0.6783 - val_loss: 0.6030 - val_accuracy: 0.6456
Epoch 

> **Nhận xét**:
> * Có thể thấy, model này hoạt động không tốt so với LSTM, trong khi với số epoch tương ứng LSTM cho ra accuracy và loss value trên validation data tốt hơn nhiều.
> * Như vậy, việc sử dụng các word vectors của fasttext không đem lại hiệu suất tốt cho tập dữ liệu của chúng ta, chúng ta có thể thay đổi kiến trúc của model để có kết quả tốt hơn, nhưng khả năng cao thì hiệu suất có thể không cải thiện $\Rightarrow$ Dừng phát triển model này.